Agents are connected but have their own states. Only the final responses are appended to global state

<img src="../../resources/images/multi_agent_supervisor.png" alt="Multi Agent Supervisor" height="300" width="300">

Agents are independent LangChain agents. This means they have their own individual prompt, LLM, and tools.

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
from src.langchain.lg_workflow import *
from src.langchain.agents.framenet_agent import *

In [2]:
fnr = framenet_tool.invoke({'instruction':'pick up the cup'})
fnr

INSIDE FRAMENET TOOL


'{\n  "framenet": "picking_up",\n  "frame": "Getting",\n  "lexical-unit": "pick up.v",\n  "core": {\n    "agent": "robot",\n    "theme_patient": "cup",\n    "instrument": "robot gripper",\n    "source": "surface",\n    "goal": "cup",\n    "result": "robot has the cup"\n  },\n  "peripheral": {\n    "location": "workspace",\n    "manner": "gently",\n    "direction": "upward",\n    "time": "during operation",\n    "quantity": "one item",\n    "portion": ""\n  }\n}'

In [3]:
# Example: Complex Query Using Multiple Agents
input_question = "framenet representation of the action pick up the mug from the sink"
for s in graph.stream(
    {"messages": [("user", input_question)]},
    subgraphs=True
):
    print(s)
    print("----")


Next Worker: framenet
((), {'supervisor': None})
----
INSIDE FRAMENET TOOL
(('framenet:4e82edc6-c0e3-8b96-d4f6-309e197939d9',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:8b', 'created_at': '2025-05-13T13:54:50.06727488Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4926080106, 'load_duration': 8142009, 'prompt_eval_count': 294, 'prompt_eval_duration': 40061265, 'eval_count': 352, 'eval_duration': 4871349509, 'model_name': 'qwen3:8b'}, id='run--238ded2f-a2bf-4642-9fbe-46f9b271adec-0', tool_calls=[{'name': 'framenet_tool', 'args': {'instruction': 'pick up the mug from the sink'}, 'id': '5f885a89-5ad9-4db2-bf24-9301530a88ee', 'type': 'tool_call'}], usage_metadata={'input_tokens': 294, 'output_tokens': 352, 'total_tokens': 646})]}})
----
(('framenet:4e82edc6-c0e3-8b96-d4f6-309e197939d9',), {'tools': {'messages': [ToolMessage(content='{\n  "framenet": "picking_up",\n  "frame": "Getting",\n  "lexical-unit": "pick up.v",\

In [12]:
graph.invoke({'messages': [HumanMessage(content='framenet representation of the action pick up the mug from the fridge')]})['messages'][0].content

Next Worker: framenet
INSIDE FRAMENET TOOL
Next Worker: FINISH


'framenet representation of the action pick up the mug from the fridge'

In [13]:
graph.invoke({'messages': [HumanMessage(content='framenet representation of the action pick up the mug from the fridge')]})['messages'][1].content

Next Worker: framenet
INSIDE FRAMENET TOOL
Next Worker: FINISH


'<think>\nOkay, the user asked for the FrameNet representation of "pick up the mug from the fridge." I called the framenet_tool with that instruction. The response came back with the frame "Getting" and the lexical unit "pick up.v". The core elements have agent as robot, theme_patient as mug, instrument as robot gripper, source as fridge, and the result is the robot having the mug. Peripheral details include location, manner, direction, etc.\n\nNow, I need to present this information as the final answer. The user probably wants a clear, structured output without extra explanations. Let me format the response using the FrameNet elements provided. Make sure to include all the core and peripheral elements as listed. Use the exact terms from the tool\'s response. Avoid any markdown and just present the data in a readable way. Check if all the keys and values are correctly transcribed. Alright, that should do it.\n</think>\n\n{\n  "framenet": "picking_up",\n  "frame": "Getting",\n  "lexical

In [2]:
framenet_agent.invoke({'messages': [HumanMessage(content='pick up the mug from the sink')]})

INSIDE FRAMENET TOOL
INSIDE FRAMENET TOOL


{'messages': [HumanMessage(content='pick up the mug from the sink', additional_kwargs={}, response_metadata={}, id='0c900108-fb9d-49c0-ad45-a73345423c53'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:8b', 'created_at': '2025-05-13T14:50:39.709835015Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11338841794, 'load_duration': 1242114656, 'prompt_eval_count': 282, 'prompt_eval_duration': 146014174, 'eval_count': 739, 'eval_duration': 9942925922, 'model_name': 'qwen3:8b'}, id='run--9ff62ee9-c613-4f62-8343-f5da287fd2bd-0', tool_calls=[{'name': 'framenet_tool', 'args': {'instruction': 'pick up the mug from the sink'}, 'id': '91594288-2c08-4c94-80a7-911b4866e791', 'type': 'tool_call'}, {'name': 'framenet_tool', 'args': {'instruction': 'pick up the mug from the sink'}, 'id': '9924449d-9bb5-4fac-8d38-1eebe6c7a96b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 282, 'output_tokens': 739, 'total_tokens': 1021}),
  ToolMessage(content='"{\\

In [ ]:
from IPython.display import display, Image
# display(Image(math_agent.get_graph().draw_mermaid_png()))
# display(Image(websearch_agent.get_graph().draw_mermaid_png()))
# display(Image(framenet_agent.get_graph().draw_mermaid_png()))
# display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Example: Complex Query Using Multiple Agents
input_question = "what is 2 times 2"
for s in graph.stream(
    {"messages": [("user", input_question)]},
    subgraphs=True
):
    print(s)
    print("----")


In [ ]:
# Example: Complex Query Using Multiple Agents
input_question = "what is the value of 2 + 10"
for s in graph.stream(
    {"messages": [("user", input_question)]},
    subgraphs=True
):
    print(s)
    print("----")


In [ ]:
# # Example: Complex Query Using Multiple Agents
# input_question = "who is apple company founder"
# for s in graph.stream(
#     {"messages": [("user", input_question)]},
#     subgraphs=True
# ):
#     print(s)
#     print("----")


In [1]:
# # Example: Complex Query Using Multiple Agents
# input_question = "who is the ceo of google and what is the framenet representation of apple"
# for s in graph.stream(
#     {"messages": [("user", input_question)]},
#     subgraphs=True
# ):
#     print(s)
#     print("----")


In [2]:
# # Example: Complex Query Using Multiple Agents
# input_question = "who is the ceo of google, what is the framenet representation of apple and how much is 2 times 10"
# for s in graph.stream(
#     {"messages": [("user", input_question)]},
#     subgraphs=True
# ):
#     print(s)
#     print("----")
